In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#mounting data from google drive
from google.colab import drive
drive.mount("/content/drive")
df=pd.read_csv("/content/drive/MyDrive/NLP Data/movie_data.csv")
#df = df.iloc[:1000]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0


In [ ]:
#Making sure all the positive and negetive reviews are equal
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
#Shows if there are duplicated entries
df.duplicated().sum()

418

In [ ]:
#Removes duplicated entries
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['review'].values[1]

"OK... so... I really like Kris Kristofferson and his usual easy going delivery of lines in his movies. Age has helped him with his soft spoken low energy style and he will steal a scene effortlessly. But, Disappearance is his misstep. Holy Moly, this was a bad movie! <br /><br />I must give kudos to the cinematography and and the actors, including Kris, for trying their darndest to make sense from this goofy, confusing story! None of it made sense and Kris probably didn't understand it either and he was just going through the motions hoping someone would come up to him and tell him what it was all about! <br /><br />I don't care that everyone on this movie was doing out of love for the project, or some such nonsense... I've seen low budget movies that had a plot for goodness sake! This had none, zilcho, nada, zippo, empty of reason... a complete waste of good talent, scenery and celluloid! <br /><br />I rented this piece of garbage for a buck, and I want my money back! I want my 2 hou

In [ ]:
import re

for i in range(len(df)):
  df['review'].values[i] = re.sub(re.compile('<.*?>'), '', df['review'].values[i])
  df['review'].values[i] = re.sub(r'[^\w\s]', '', df['review'].values[i])

In [ ]:
#Makes all the entries in reviews lower case
df['review'] = df['review'].apply(lambda x:x.lower())

In [ ]:
#using NLTK we remove the  stop words inside dataframe
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['review'].values[2]

'spoiler read think watching movie although would waste time way plot predictable make difference read anywayif wondering whether see coyote ugly dont worth either money ticket vhs dvd typical chickfeelgoodflick one could say plot shallow ridiculous uncritical version american dream young goodlooking girl small town becoming big success new york desperate attempts giving movie depth fail tragic accident father difficulties violets relationship boyfriend mcnally director tries arouse audiences pity sadness put chance succeed attempt due bad script shallow acting especially piper perabo completely fails convincing one jerseys fear singing front audience good quite funny thing coyote ugly john goodman represents small ray hope moviei astonished jerry bruckheimer produced movie first gone 60 seconds happened great movies like rock con air true bruckheimer stuffif looking superficial movie good looking women relaxed evening better go see charlies angels much funny entertaining selfironic in

In [ ]:
#gensim is used to create our model
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
#coverts the data into tokens and using gensim we make the tokens go through another preprocess
nltk.download('punkt')
x_tokens = []
for review in df['review']:
  raw_sent = sent_tokenize(review)
  for sent in raw_sent:
    x_tokens.append(simple_preprocess(sent))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
(x_tokens)

In [ ]:
res = [ele for ele in x_tokens if ele != []]
x_tokens = res
len(x_tokens)

49582

In [ ]:
#We use the Word2Vec model of gensim as the model is trained to create word embeddings

model = gensim.models.Word2Vec(
    window=10, #The window parameter specifies the maximum distance between the current and predicted word within a sentence.
    min_count=2 #The min_count parameter ignores all words with total frequency lower than this
)

In [ ]:
model.build_vocab(x_tokens)

In [ ]:
model.train(x_tokens, total_examples=model.corpus_count, epochs=model.epochs)

(27672164, 29333995)

In [ ]:
#Total vocabulary of this corpus
len(model.wv.index_to_key)

79977

In [ ]:
#This function converts the reviews into word vectors that will be used as inputs for the classfier training and testing
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
from tqdm import tqdm

In [ ]:
x = []
for doc in tqdm(df['review'].values):
    x.append(document_vector(doc))

100%|██████████| 49582/49582 [44:09<00:00, 18.72it/s]


In [ ]:
x = np.array(x)
x.shape


(49582, 100)

In [ ]:
y = df['sentiment']

In [ ]:
#Splitting of data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)


In [ ]:
#Using the Random Forest Classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Training of the classifier and calculation of the results
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print(accuracy_score(y_test,y_pred)*100)

84.64253302410003


In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(x_train, y_train)
y_pred = knn_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy*100)

81.72834526570536


In [ ]:
custom_input = "Man! This was was GREAT! So should see it with your family, friends AND Ex-Wife!"
custom_input = document_vector(custom_input)
predicted_label = rf.predict([custom_input])

if predicted_label[0] == 1:
  print("Positive")
else:
  print("Negetive")

Positive


In [ ]:
custom_input = "Man! This movie was terrible! Never watch it"
custom_input = document_vector(custom_input)
predicted_label = knn_classifier.predict([custom_input])

if predicted_label[0] == 1:
  print("Positive")
else:
  print("Negetive")

Negetive
